# ODE

We will solve the following linear Cauchy model

\begin{align}
y^{\prime}(t) &= \lambda y(t)\\
y(0) & = 1
\end{align}

whose exact solution is

$$
y(t) = e^{\lambda t}
$$


In [ ]:
%matplotlib inline
from numpy import *
from matplotlib.pyplot import *
import scipy.linalg
import numpy.linalg

l = -5.
t0 = 0.
tf = 10.
y0 = 1.

s = linspace(t0,tf,5000)

exact = lambda x: exp(l*x)



### Forward Euler

$$
\frac{y_{n}-y_{n-1}}{h} = f(y_{n-1}, t_{n-1})
$$

- Is this method implicit or explicit?
- Try to solve the ODE with the following step size: 0.9, 0.41, 0.39, 0.15
- Check the order of convergence

In [ ]:
def fe(l,y0,t0,tf,h):


### Backward Euler

$$
\frac{y_{n}-y_{n-1}}{h} = f(y_{n}, t_{n})
$$

- Is this method implicit or explicit?
- Try to solve the ODE with the following step size: 0.9, 0.41, 0.39, 0.15
- Check the order of convergence

In [ ]:
def be(l,y0,t0,tf,h):


### $\theta$-method

$$
\frac{y_{n}-y_{n-1}}{h} = \theta\, f(y_{n}, t_{n}) + (1-\theta)\,f(y_{n-1}, t_{n-1})
$$

- Is this method implicit or explicit?
- Fix $h=0.39$ and use $\theta =0, 1, 0.5$. What do you see?
- Check the order of convergence for $\theta = 0.5$

In [ ]:
def tm(theta,l,y0,t0,tf,h):


### Simple adaptive time stepper

For each time step:
- Compute solution with CN
- Compute solution with BE
- Check the difference
- If the difference satisfy a given tolerance:
    - keep the solution of higher order
    - double the step size
    - go to the next step
- Else:
    - half the step size and repeat the time step

In [ ]:
def adaptive(l,y0,t0,tf,h0, hmax=0.9,tol=1e-3):
